<a href="https://colab.research.google.com/github/HungHoangDinh/Group_Tasks_MFEAs/blob/main/Group_Task_MFEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
from sklearn.cluster import KMeans
import math

In [11]:
dimension=30
population_size=250
generate=30
O_1 = np.random.rand(dimension)
O_2 = np.random.rand(dimension)
O_3=np.random.rand(dimension)
O_4=np.random.rand(dimension)
O_5=np.random.rand(dimension)
O_6=np.random.rand(dimension)
O=[O_1,O_2,O_3,O_4,O_5,O_6]

In [3]:
def griewank(x,O):
    x=x-O;
    x=x*1200-600
    dimension = len(x)
    sum_sq = np.sum(x**2)
    prod_cos = np.prod(np.cos(x / np.sqrt(np.arange(1, dimension + 1))))
    result = 1 + (1 / 4000) * sum_sq - prod_cos
    return result
def ackley(x,O):
    x=x-O;
    x=x*64-32
    dimension = len(x)
    sum_sq = np.sum(x**2)
    cos_term = np.sum(np.cos(2 * np.pi * x))
    result = -20 * np.exp(-0.2 * np.sqrt(sum_sq / dimension)) - np.exp(cos_term / dimension) + 20 + np.exp(1)
    return result

def rastrigin(x,O):
    x=x-O
    x=x*10-5
    A = 10
    dimension = len(x)
    result = A * dimension + np.sum(x**2 - A * np.cos(2 * np.pi * x))
    return result

def weierstrass(x,O, a=0.5, b=3, kmax=20):
    x=x-O
    x=x-0.5
    dimension = len(x)
    result = 0
    for k in range(kmax + 1):
        result += np.sum(a**k * np.cos(2 * np.pi * b**k * (x + 0.5)))
    result -= dimension * kmax * np.sum(a**k)
    return result

In [4]:
functions=["griewank","ackley","rastrigin","weierstrass"]
Tasks=[]
for i in range(50):
  Tasks.append({"function": np.random.choice(functions), "O": O[random.randint(0, 5)]})


In [5]:
def initialize_population(population_size, dimension):
    return np.random.uniform(0, 1, size=(population_size, dimension))

In [6]:
def cost(task,O,population):
    result = []
    for x in population:
      result.append(task(x,O));
    return np.array(result)

In [7]:
def get_best_indivial(task,O,population):
    values=[]
    if(task=="griewank"):
      values=cost(griewank,O,population)
    if(task=="ackley"):
      values=cost(ackley,O,population)
    if(task=="rastrigin"):
      values=cost(rastrigin,O,population)
    if(task=="weierstrass"):
      values=cost(weierstrass,O,population)
    sorted_indice= np.argsort(values)
    selected_indice = sorted_indice[0]
    selected_point = population[selected_indice]
    return selected_point


In [8]:
def best_invidial_matrix(population):
  max_init_Tasks=[]
  for i in range(50):
    task=Tasks[i]["function"]
    O_task=Tasks[i]["O"]
    max_init_Tasks.append(get_best_indivial(task,O_task,population))
  return np.array(max_init_Tasks)

In [9]:


def bisecting_kmeans(data, k):
    clusters = [np.arange(len(data))]
    while len(clusters) < k:
        # Chọn cluster có độ lệch chuẩn lớn nhất để chia
        max_deviation_cluster_index = np.argmax([np.std(data[cluster]) for cluster in clusters])
        max_deviation_cluster = data[clusters[max_deviation_cluster_index]]

        # Sử dụng K-means để chia cluster lớn thành 2 cluster con
        kmeans = KMeans(n_clusters=2)
        kmeans.fit(max_deviation_cluster)

        # Cập nhật danh sách cluster
        new_cluster_indices = np.array_split(clusters[max_deviation_cluster_index], 2)
        clusters.pop(max_deviation_cluster_index)
        clusters.extend(new_cluster_indices)

    return clusters


In [10]:
population=initialize_population(population_size,dimension)
BI=best_invidial_matrix(population)

In [12]:
def invidial_cost(task,O,invidiual):
    values=0
    if(task=="griewank"):
      values=griewank(invidiual,O)
    if(task=="ackley"):
      values=ackley(invidiual,O)
    if(task=="rastrigin"):
      values=rastrigin(invidiual,O)
    if(task=="weierstrass"):
      values=weierstrass(invidiual,O)
    return values

In [13]:
def grouping_task(BI):
   clusters = bisecting_kmeans(BI, 4)
   newclusters=clusters
   k=int(len(clusters)/2)
   RG=[]
   for i in range(len(clusters)):
      RG.append({"a":clusters[i][0],"b":i})
   kRG=random.sample(RG, k)
   t=k
   ERG=[]
   for i in range(k):
    tRG=random.sample(RG,t)
    f=Tasks[kRG[i]["a"]]["function"]
    O=Tasks[kRG[i]["a"]]["O"]
    value=invidial_cost(f,O,BI[clusters[kRG[i]["b"]][1]])
    cluster=kRG[i]["b"]
    for x in tRG:
      if(invidial_cost(f,O,BI[x["a"]])>value):

        value=invidial_cost(f,O,BI[x["a"]])
        cluster=x["b"]
    if(cluster!=kRG[i]["b"]):
        invidiual=kRG[i]["a"]
        for j in range(len(newclusters)):
          if invidiual in newclusters[j]:
            newclusters[j].remove(invidiual)
          if(j==cluster):
            newclusters[j].append(invidiual)
   return newclusters





In [15]:
group=grouping_task(BI)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [16]:
def FS(task,O,invidiual,population):
  max_f=0;

  value=invidial_cost(task,O,invidiual)
  min_f=invidial_cost(task,O,population[0])
  for i in range(len(population)):
    a=invidial_cost(task,O,population[i])
    if(max_f<a):
      max_f=a
    if(min_f>a):
      min_f=a
  return (value-min_f)/(max_f-min_f)

In [17]:
def CD(invidiual,population):
  neighbor_size=0.4
  tong=0
  for i in range(len(population)):
    absolute_sum = np.sum(np.abs(invidiual-population[i]))
    if (absolute_sum<0.4):
      tong=tong+(1-absolute_sum)
  return tong


In [18]:
def mating_selection(group_task,population_,alpha):
  print(len(population_))
  np.random.shuffle(population_)
  NP=[]
  for i in range(len(group_task)):
    index_task=group_task[i][0]
    f=Tasks[index_task]["function"]
    O=Tasks[index_task]["O"]
    for j in range(int(len(population)/2)):
      p1=population[2*j]
      p2=population[2*j+1]
      if ((alpha*FS(f,O,p1,population_)+(1-alpha)*CD(p1,population_))<(alpha*FS(f,O,p1,population_)+(1-alpha)*CD(p1,population_))):
        if not any(np.array_equal(p1, x) for x in NP):
            NP.append(p1)
      else:
        if not any(np.array_equal(p2, x) for x in NP):
            NP.append(p2)
  return NP

In [19]:
print(len(population))
population=mating_selection(group,population,0.95)

250
250


In [20]:
def get_rank(population):
  rank=[]
  for i in range(len(Tasks)):
    f=Tasks[i]["function"]
    O=Tasks[i]["O"]
    value_task=[]
    for j in range(len(population)):
      invidiual=population[j]
      value=invidial_cost(f,O,invidiual)
      value_task.append(value)
    rank.append(np.argsort(np.argsort(value_task)))
  return rank


In [ ]:
print(get_rank(population))

In [22]:
def get_skill_factor(population):
  rank=get_rank(population)
  skill_factor=[]
  for i in range(len(population)):
    skill=0
    rank_i=rank[0][i]
    for j in range(len(rank)):
      if(rank[j][i]>rank_i):
        skill=j
        rank_i=rank[j][i]
    skill_factor.append(skill)
  return skill_factor
skill_factor=get_skill_factor(population)

In [23]:
def crossover(parent1, parent2):
    cf=random.random()
    child1 =0.5*( parent1 * (1+cf) + parent2 * (1 - cf))
    child2 =0.5*( parent2 * (1+cf) + parent1 * (1 - cf))
    return child1, child2

In [24]:
def mutation(individual, mutation_rate, mutation_scale):
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if (np.random.rand() < mutation_rate ):
            mutated_individual[i] += np.random.normal(scale=mutation_scale)
    return mutated_individual

In [ ]:
print(group)

In [26]:
def check_group(skill_factor_1,skill_factor_2):
   for i in range(len(group)):
    if skill_factor_1 in group[i] and skill_factor_2 in group[i]:
      return True
   return False


In [27]:
def assortative_mating(p1,p2, skill_factor1,skill_factor2):
    rmp=0.3
    rnd = np.random.rand(1)[0]
    if skill_factor1== skill_factor2 or(rnd<0.8 and check_group(skill_factor1,skill_factor2)) or rnd < rmp:
      o1,o2=crossover(p1,p2);
      random_skill_factor = np.random.rand(1)[0]
      if random_skill_factor < 0.5:
        return o1,o2,skill_factor1,skill_factor1
      else:
        return o1,o2,skill_factor2,skill_factor2
    else:
        o1= p1.copy()
        o2 = p2.copy()
        o1=mutation(o1,0.1,0.1);
        o2=mutation(o2,0.1,0.1);
        return o1,o2,skill_factor1,skill_factor2

In [28]:
def update_CD(population,select_invi,CD):
  neighbor_size=0.4

  for i in range(len(population)):
    invidiual=population[i]
    absolute_sum = np.sum(np.abs(invidiual-select_invi))
    if (absolute_sum<0.4):
      CD[i]=CD[i]+1-absolute_sum
  return CD

In [29]:
def envi_selection(population,Tasks,skill_factor,alpha):
  NP=[]
  NK=[]
  rate=125/len(population)
  CD_array=np.zeros(len(population))
  for i in range(len(Tasks)):
    P_1=population[np.where(skill_factor==i)[0]]
    new_CD=CD_array[np.where(skill_factor==i)[0]]
    tP=[]
    while len(tP)<math.ceil(rate*len(P_1)):
      q=P_1[0]
      value=alpha*FS(Tasks[i]["function"],Tasks[i]["O"],q,population)+(1-alpha)*CD(q,population)
      for j in range(len(P_1)):
        invi=P_1[j]

        if(alpha*FS(Tasks[i]["function"],Tasks[i]["O"],invi,population)+(1-alpha)*new_CD[j]<value):
          value=alpha*FS(Tasks[i]["function"],Tasks[i]["O"],invi,population)+(1-alpha)*new_CD[j]
          q=invi
      tP.append(q)
      CD_array=update_CD(population,q,CD_array)
      new_CD=update_CD(P_1,q,new_CD)
    for h in range(len(tP)):
      NP.append(tP[h])
    NK=np.concatenate((NK,[i]*len(tP)))
  return NP,NK




In [30]:
def Group_MFEA(population,skill_factor):
  for i in range(generate):
    offspring=[]
    skill_factor_offspring=[]
    for j in range(math.ceil(len(population)/2)):
       random_indices = np.random.choice(np.arange(len(population)), size=2, replace=False)
       p1,p2=population[random_indices[0]],population[random_indices[1]]
       o1,o2,skill_factor1,skill_factor2=assortative_mating(p1,p2,skill_factor[random_indices[0]],skill_factor[random_indices[1]])
       offspring.append(o1)
       offspring.append(o2)
       skill_factor_offspring.append(skill_factor1)
       skill_factor_offspring.append(skill_factor2)
    intermediate_pop = np.vstack((population, offspring))
    intermediate_skill_factor = np.concatenate((skill_factor,skill_factor_offspring))
    population, skill_factor = envi_selection(intermediate_pop,Tasks, intermediate_skill_factor,0.95)
    print(len(population))
  newBI=best_invidial_matrix(population)
  for i in range(len(newBI)):
    f=Tasks[i]["function"]
    O=Tasks[i]["O"]
    value=invidial_cost(f,O,BI[i])
    print(f"Task {i}: {value}")

In [31]:
Group_MFEA(population,skill_factor)

131
133
135
134
135
135
134
134
135
134
133
135
135
133
136
133
135
134
135
135
134
135
135
135
136
136
134
135
136
135
Task 0: 3130.119816705758
Task 1: 21.335846245555818
Task 2: 21.178867797582875
Task 3: 3170.4189091049448
Task 4: -12.102199741018097
Task 5: 2544.905153493727
Task 6: 21.212397493222987
Task 7: 904.544383354117
Task 8: 2338.6191200785574
Task 9: -12.102199741018097
Task 10: -13.339240160251828
Task 11: 2338.6191200785574
Task 12: 904.544383354117
Task 13: 2544.905153493727
Task 14: 2665.5380786914734
Task 15: 2338.6191200785574
Task 16: 1047.6770800707054
Task 17: -13.237003742798365
Task 18: 1047.6770800707054
Task 19: -13.236306669472654
Task 20: 21.178867797582875
Task 21: 2665.5380786914734
Task 22: 1047.6770800707054
Task 23: -13.236306669472654
Task 24: 946.1582591472381
Task 25: 21.178867797582875
Task 26: 2338.6191200785574
Task 27: 3170.4189091049448
Task 28: -13.237003742798365
Task 29: 1051.5447343781682
Task 30: -12.102199741018097
Task 31: -13.237003742